In [14]:
prom_file_path = "../prom_integrated.parquet"

In [8]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras import regularizers

In [15]:
import pandas as pd
prom_df = pd.read_parquet(prom_file_path)

In [4]:
prom_df

,timestamp,node,node_load15,node_load5,node_load1,node_memory_Active_bytes,node_filesystem_device_error-sum,node_netstat_Icmp_InErrors,node_netstat_Tcp_InErrs,node_netstat_Udp_InErrors,nvidia_gpu_power_usage_milliwatts-max,nvidia_gpu_temperature_celsius-max,isAnomaly,__index_level_0__,jobID,finishTasks,jobCount
0,2022-06-30 16:00:30,r11n18,1.84,1.92,2.00,2.534220e+10,0.0,726.0,0.0,0.0,0.0,0.0,0,20350633,[],NaN,0
1,2022-06-30 16:00:30,r27n11,16.04,16.04,16.02,6.771600e+09,0.0,2.0,0.0,0.0,0.0,0.0,0,105359547,[],NaN,0
2,2022-06-30 16:01:00,r10n32,16.06,16.07,16.17,1.663300e+10,0.0,0.0,0.0,0.0,0.0,0.0,0,13026555,[],NaN,0
3,2022-06-30 16:01:00,r25n4,12.63,13.00,12.78,3.813450e+09,0.0,713.0,0.0,0.0,0.0,0.0,0,77281352,[],NaN,0
4,2022-06-30 16:01:00,r25n6,16.04,16.07,16.21,1.313900e+10,0.0,0.0,0.0,0.0,0.0,0.0,0,78092172,[],NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104354189,2022-10-31 23:00:00,r10n28,16.06,16.03,16.00,1.927770e+09,0.0,0.0,0.0,0.0,0.0,0.0,0,11637548,[],NaN,0
104354190,2022-10-31 23:00:00,r12n24,16.06,16.03,16.00,1.800450e+10,0.0,0.0,0.0,0.0,0.0,0.0,0,35628436,[],NaN,0
104354191,2022-10-31 23:00:00,r12n32,0.08,0.18,0.05,1.159850e+09,0.0,0.0,0.0,0.0,0.0,0.0,0,38473035,[],NaN,0
104354192,2022-10-31 23:00:00,r25n8,16.05,16.02,16.00,1.651980e+09,0.0,0.0,0.0,0.0,0.0,0.0,0,79149353,[],NaN,0


In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
prom_df.fillna(0, inplace=True)
feature_cols = [col for col in prom_df.columns if col not in ['node', 'timestamp', 'isAnomaly','node_netstat_Udp_InErrors','node_netstat_Tcp_InErrs','node_netstat_Icmp_InErrors','node_filesystem_device_error-sum','jobID']]
X = prom_df[feature_cols]
y = prom_df['isAnomaly']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

input_dim = X.shape[1]  
encoding_dim = 14  
batch_size = 32
epochs = 10

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer=Adam(), 
                    loss='mean_squared_error', 
                    metrics=['mae'])

checkpoint = ModelCheckpoint('slurmmodel-{epoch:03d}.h5',  
                             monitor='val_loss',  
                             verbose=0,  
                             save_best_only=True,  
                             mode='auto')  

history = autoencoder.fit(X_scaled, X_scaled,
                          epochs=epochs,
                          batch_size=batch_size,
                          shuffle=True,
                          validation_data=(X_scaled, X_scaled),
                          callbacks=[checkpoint])  

Epoch 1/10
3261069/3261069 [==============================] - 15118s 5ms/step - loss: 0.2548 - mae: 0.2672 - val_loss: 0.2505 - val_mae: 0.2632
Epoch 2/10
3261058/3261069 [============================>.] - ETA: 0s - loss: 0.2498 - mae: 0.2638

: 

: 

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
prom_df.fillna(0, inplace=True)
feature_cols = [col for col in prom_df.columns if col not in ['node', 'timestamp', 'isAnomaly','node_netstat_Udp_InErrors','node_netstat_Tcp_InErrs','node_netstat_Icmp_InErrors','node_filesystem_device_error-sum','jobID']]
X = prom_df[feature_cols]
y = prom_df['isAnomaly']
autoencoder = load_model('slurmmodel-001.h5')
input_dim = X.shape[1]  
encoding_dim = 14  
batch_size = 32
epochs = 5

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

checkpoint = ModelCheckpoint('slurmmodel-{epoch:03d}.h5',  
                             monitor='val_loss',  
                             verbose=0,  
                             save_best_only=True,  
                             mode='auto')  

history = autoencoder.fit(X_scaled, X_scaled,
                          epochs=epochs,
                          batch_size=batch_size,
                          shuffle=True,
                          validation_data=(X_scaled, X_scaled),
                          callbacks=[checkpoint])  


Epoch 1/5
32579/32579 [==============================] - 120s 4ms/step - loss: 0.2505 - mae: 0.2647 - val_loss: 0.2495 - val_mae: 0.2625
Epoch 2/5
12413/32579 [==========>...................] - ETA: 51s - loss: 0.2500 - mae: 0.2641

KeyboardInterrupt: 

In [16]:
from tensorflow.keras.models import load_model
feature_cols = [col for col in prom_df.columns if col not in ['node', 'timestamp', 'isAnomaly','node_netstat_Udp_InErrors','node_netstat_Tcp_InErrs','node_netstat_Icmp_InErrors','node_filesystem_device_error-sum','jobID']]
X = prom_df[feature_cols]
y = prom_df['isAnomaly']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

input_dim = X.shape[1]  
encoding_dim = 14  
batch_size = 32
epochs = 10
autoencoder = load_model('slurmmodel-001.h5')

predictions = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - predictions, 2), axis=1)

threshold = np.quantile(mse, 0.972)  

y_pred = [1 if e > threshold else 0 for e in mse]

print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))

2608841/2608841 [==============================] - 2239s 858us/step


/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

              precision    recall  f1-score   support

           0       0.97      1.00      0.99  81348584
           1       0.00      0.00      0.00   2134311

    accuracy                           0.97  83482895
   macro avg       0.49      0.50      0.49  83482895
weighted avg       0.95      0.97      0.96  83482895

[[81348584        0]
 [ 2134311        0]]


: 

In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
feature_cols = [col for col in prom_df.columns if col not in ['node', 'timestamp', 'isAnomaly','node_netstat_Udp_InErrors','node_netstat_Tcp_InErrs','node_netstat_Icmp_InErrors','node_filesystem_device_error-sum','jobID']]
autoencoder = load_model('slurmmodel-001.h5')
batch_size = 32  
num_last_batches = 12  

start_index = -batch_size * num_last_batches
X = prom_df[feature_cols]
y = prom_df['isAnomaly']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
last_batches_X = X_scaled[start_index:]
last_batches_y = y[start_index:]

try:
    output = autoencoder.predict(last_batches_X)
    print("Model runs successfully on the last batches.")
except Exception as e:
    print("Model fails to run on the last batches. The error message is: ", e)


12/12 [==============================] - 0s 2ms/step
Model runs successfully on the last batches.


In [ ]:
predictions = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - predictions, 2), axis=1)

threshold = np.quantile(mse, 0.972)  

y_pred = [1 if e > threshold else 0 for e in mse]

print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))

NameError: name 'autoencoder' is not defined